__Code to create a random distribution of atoms around two hollow spheres__

    - The hollow spheres will be the sites for crystal seeds
    - RUNTIME for this code will depend on cut-off values used
        - higher cut-off, lower RUNTIME and vice-versa

In [1]:
import numpy as np

In [2]:
def periodic_distance(atom, nbd, box):
    '''Inputs:
        atom = coordinate of a single reference atom
        nbd = array of coordinates of multiple atoms
        box = length of the box
       Outputs:
        dis = distance between atom and nbd
        len(dis) == len(nbd)
    '''
    delta = np.abs(atom-nbd)
    delta = np.where(delta> 0.5*box, delta-box, delta)
    dis = np.sqrt((delta**2).sum(axis=-1))
    return dis

In [3]:
import periodictable

def get_atomic_mass(element_symbol):
    try:
        element = periodictable.elements.symbol(element_symbol)
        atomic_mass = element.mass
        return atomic_mass
    except ValueError as e:
        print(f"Error: {e}")
        return None


In [4]:
# Choose an x for creating (1-x)LiNbO3-xSiO2 system
frac = 0.0

# We want a total of n_mols number of molecules (sum of number of LiNbO3 and SiO2 molecules)
# If x == 0, there will be n_mols number of LiNbO3
n_mols = 40000
n_Li = int(n_mols*(1-frac))
n_Nb = n_Li

n_O = int(n_mols*(frac*2+(1-frac)*3))
n_Si = int(n_mols*frac)

# # Manually choose the number of Er
# n_Er = 450
# n_O += int(n_Er*3/2.0)

print("Number of atoms: Li:{}, Nb:{}, O:{}, Si:{}".format(n_Li, n_Nb, n_O, n_Si))
print("Total number of atoms: ", np.sum([n_Li, n_Nb, n_O, n_Si]))

Number of atoms: Li:40000, Nb:40000, O:120000, Si:0
Total number of atoms:  200000


In [5]:
contents = {'Li':n_Li,'Nb':n_Nb,'O':n_O}#,'Si':n_Si,'Er':n_Er} #LNS5

total_mass = 0.0
for key,num in contents.items():
    total_mass += num*get_atomic_mass(key)

In [6]:
import numpy as np


density = 4.09   #g/cm3
box = np.cbrt((total_mass*1.66054)/(density))
print("Box size:", box, "A.")

Box size: 133.90540995100653 A.


__Choosing the center of the seeds__

    - distance between the seeds (15 A and 10 A) in the earlier class of simulation = 50 A
    - closest distance between surface = 25 A
    - To maintain the same condition, two 15 A seeds centers must be 55 A away

In [7]:
rad_I, rad_II = 15.0, 15.0

c_I = [box/2+16.0, box/2-16.0, box/2+16.0]
c_II = [box/2-16.0, box/2+15.0, box/2-16.0]
d_seed = np.sqrt(sum((x - y) ** 2 for x, y in zip(c_I,c_II)))
print("Seed Centers:")
print(c_I)
print(c_II)
print("Distance between seed centers: ",np.sqrt(sum((x - y) ** 2 for x, y in zip(c_I,c_II))))
print("Closest approach of seed surfaces is {}.".format(d_seed-(rad_I+rad_II)))
def isInSphere(_x,_y,_z):
    '''
    Takes a specified co-ordinate and determine if the points fall within the spherical constrains
    Returns one that works if the answer is FALSE
    '''
    ## if point is inside circle reject it
    # add 1.0 to make sure atoms are not overlapping when the seeds and the sorrounding are later placed together
    while (_x - c_I[0])**2 + (_y - c_I[1])**2 + (_z - c_I[2])**2 < (rad_I+1.0)**2 \
        or (_x - c_II[0])**2 + (_y - c_II[1])**2 + (_z - c_II[2])**2 < (rad_II+1.0)**2:        
        _x = box*ran.random()
        _y = box*ran.random()
        _z = box*ran.random()        
    return _x,_y,_z 

Seed Centers:
[82.95270497550327, 50.952704975503266, 82.95270497550327]
[50.952704975503266, 81.95270497550327, 50.952704975503266]
Distance between seed centers:  54.85435260760991
Closest approach of seed surfaces is 24.85435260760991.


In [8]:
num_atoms = list(contents.values())

In [9]:
natoms_summed = [sum(num_atoms[:i+1]) for i in range(len(num_atoms))]

print(natoms_summed)


[40000, 80000, 200000]


__Want to Check if the code is working with a small number of atoms?__

Uncomment the next code snippet

In [10]:
#natoms_summed = [4000,8000,20000]

In [11]:
from tqdm import tqdm
import random as ran
import numpy as np



cut_off = 1.5                 #minimum approach distance
center=[box*0.5, box*0.5, box*0.5]
rx,ry,rz = [],[],[]
print ("Now creating the random coordinates.................")


print ("Now placing Li!!!")
for count in tqdm(range(natoms_summed[0]), desc="Processing", ncols=100):
    _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
    x,y,z = isInSphere(_x,_y,_z)
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+1.0 for value in dist):         #Li atoms are no closer than 2.50 A 
            checker = "Passed"
        else:
            _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
            x,y,z = isInSphere(_x,_y,_z)
    rx.append(x)
    ry.append(y)
    rz.append(z)


Now creating the random coordinates.................
Now placing Li!!!


Processing: 100%|█████████████████████████████████████████████| 40000/40000 [07:42<00:00, 86.41it/s]


In [12]:
print ("Now placing Nb")
for count in tqdm(np.arange(natoms_summed[0],natoms_summed[1]), desc="Processing", ncols=100):
    _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
    x,y,z = isInSphere(_x,_y,_z)
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+1.0 for value in dist):  #Li-Nb, Nb-Nb are no closer than 2.50 A
            checker = "Passed"
        else:
            _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
            x,y,z = isInSphere(_x,_y,_z)
    rx.append(x)
    ry.append(y)
    rz.append(z)

Now placing Nb


Processing: 100%|███████████████████████████████████████████| 40000/40000 [5:35:31<00:00,  1.99it/s]


In [13]:
len(rx)

80000

In [14]:
print ("Now placing O")
for count in tqdm(np.arange(natoms_summed[1],natoms_summed[2]), desc="Processing", ncols=100):
    _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
    x,y,z = isInSphere(_x,_y,_z)
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off for value in dist):
            checker = "Passed"
        else:
            _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
            x,y,z = isInSphere(_x,_y,_z)
    rx.append(x)
    ry.append(y)
    rz.append(z)

Now placing O


Processing: 100%|█████████████████████████████████████████| 120000/120000 [4:20:55<00:00,  7.66it/s]


print ("Now placing Si")
for count in tqdm(np.arange(natoms_summed[2],natoms_summed[3]), desc="Processing", ncols=100):
    _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
    x,y,z = isInSphere(_x,_y,_z)
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+0.3 for value in dist):
            checker = "Passed"
        else:
            _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
            x,y,z = isInSphere(_x,_y,_z)
    rx.append(x)
    ry.append(y)
    rz.append(z)

print ("Now placing Er")
for count in tqdm(np.arange(natoms_summed[3],natoms_summed[4]), desc="Processing", ncols=100):
    _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
    x,y,z = isInSphere(_x,_y,_z)
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+0.3 for value in dist):
            checker = "Passed"
        else:
            _x,_y,_z = box*ran.random(),box*ran.random(),box*ran.random()
            x,y,z = isInSphere(_x,_y,_z)
    rx.append(x)
    ry.append(y)
    rz.append(z)

In [15]:
out2 = open("LNO"+str(int(frac*100))+"_random.POSCAR", 'w')
print ("comment line",file=out2)
print ("%16.10f"%(1.0),file=out2)
print ("%23.16f"%(box),"%22.16f"%(0.0),"%22.16f"%(0.0),file=out2)
print ("%23.16f"%(0.0),"%22.16f"%(box),"%22.16f"%(0.0),file=out2)
print ("%23.16f"%(0.0),"%22.16f"%(0.0),"%22.16f"%(box),file=out2)
print ("   Li","   Nb","    O",file=out2)
print ("   ",n_Li,"   ",n_Nb,"    ",n_O,file=out2)
print ("Cartesian",file=out2)

    
for ct in range(len(rx)):
    print ("%20.16f"%(rx[ct]),"%20.16f"%(ry[ct]),"%20.16f"%(rz[ct]),file=out2)
    
out2.close()

out3 = open("LNS"+str(int(frac*100))+"_random.POSCAR", 'w')
print ("comment line",file=out3)
print ("%16.10f"%(1.0),file=out3)
print ("%23.16f"%(box),"%22.16f"%(0.0),"%22.16f"%(0.0),file=out3)
print ("%23.16f"%(0.0),"%22.16f"%(box),"%22.16f"%(0.0),file=out3)
print ("%23.16f"%(0.0),"%22.16f"%(0.0),"%22.16f"%(box),file=out3)
print ("   Li","   Nb","    O","    Si",file=out3)
print ("   ",n_Li,"   ",n_Nb,"    ",n_O-int(n_Er*3/2.0),"    ",n_Si,file=out3)
print ("Cartesian",file=out3)

rx = rx[:5*n_Li]+rx[5*n_Li+int(n_Er*3/2.0):]
ry = ry[:5*n_Li]+ry[5*n_Li+int(n_Er*3/2.0):]
rz = rz[:5*n_Li]+rz[5*n_Li+int(n_Er*3/2.0):]
print(len(rx), len(ry), len(rz))
for ct in range(len(rx)-n_Er):
    print ("%20.16f"%(rx[ct]),"%20.16f"%(ry[ct]),"%20.16f"%(rz[ct]),file=out3)
out3.close()